In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import time

# Define el alcance y carga las credenciales, en este caso autorizamos al ingreso de nuestra cuenta de 
#Googledrive y de SpreadSheets(excel), utilizando el json obtenido de la API de googledrive
scope = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
credenciales = ServiceAccountCredentials.from_json_keyfile_name('cred.json', scope)

# Autorizamos el cliente
cliente = gspread.authorize(credenciales)

# Abrimos la hoja de cálculo por su nombre
sheet = cliente.open("Base de datos con ficha de propietarios")  # Nombre del archivo de la hoja de cálculo

# Selecciona la worksheet por su nombre
worksheet = sheet.worksheet("Registro de Pagos")  # Nombre de la pestaña (worksheet)

# Lee los datos actuales para conocer el número de registros
data = worksheet.get_all_values()
df = pd.DataFrame(data[1:], columns=data[0])  # Carga los datos actuales en un DataFrame

# Cargamos el DataFrame desde el archivo Excel
pagos_rec = pd.read_excel('pagos_reconocidos.xlsx')

# Seleccionamos las columnas que necesitas agregar
nuevo_df = pagos_rec[['propietario', 'Pago', 'Fecha']]

# Cambiamos los nombres de las columnas para que coincidan con las de la worksheet
nuevo_df.columns = ['Nombre del Propietario', 'Monto Pagado', 'Fecha de Pago']

# Conviertimos el DataFrame en una lista de listas para agregar a Google Sheets
filas_para_agregar = nuevo_df.values.tolist()

# Ajustamos el tamaño del lote, para ir cargando en partes y no sobrepasar la cantidad diaria gratuita que permite el uso de la API 
tamano_lote = 10

# Encuentra la última fila ocupada en la hoja de cálculo
ultima_fila = len(data)  # La última fila que ya tiene datos

# Agregamos las filas en lotes para evitar exceder la cuota, tambien agregamos un try/except para entender si se estan cargando o hay algun error.
total_filas = len(filas_para_agregar)
for i in range(0, total_filas, tamano_lote):
    lote = filas_para_agregar[i:i + tamano_lote]
    try:
        # Calcula la fila inicial para el lote actual
        fila_inicial = ultima_fila + 1
        fila_final = fila_inicial + len(lote) - 1
        
        # Definimos el rango de celdas para agregar las nuevas filas en las columnas B, C y D
        rango = f"B{fila_inicial}:D{fila_final}"
        
        # Actualiza las celdas en el rango definido
        worksheet.update(range_name=rango, values=lote)
        
        print(f"Se han agregado {len(lote)} registros al rango {rango}.")
        
        # Actualiza la última fila ocupada
        ultima_fila += len(lote)  
    except Exception as e:
        print(f"Error al agregar registros: {e}")
    
    time.sleep(1)  # Espera un segundo entre cada lote para no exceder el límite de solicitudes por minuto

print(f"Se han agregado {total_filas} registros en total a la hoja de cálculo.")


Se han agregado 10 registros al rango B1012:D1021.
Se han agregado 10 registros al rango B1022:D1031.
Se han agregado 20 registros en total a la hoja de cálculo.
